In [1]:
pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Structured Streaming 

In [19]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

# 필요한 패키지 설정
spark_version = '3.5.2'  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{},org.apache.kafka:kafka-clients:2.8.1 pyspark-shell'.format(spark_version)

# Kafka 설정 정보
kafka_bootstrap_servers = 'localhost:9092'  
topic = 'new_test_topic'

# Spark 세션 생성
spark = SparkSession.builder \
    .appName("KafkaStructuredStreaming") \
    .getOrCreate()

# Kafka에서 메시지 수신
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load()


In [20]:
# 실시간으로 메시지 개수 카운트
count_df = df.groupBy().agg(count("value").alias("message_count"))

# 5초마다 카운트 값을 콘솔에 출력
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime='5 seconds') \
    .start()
# 스트리밍 실행 대기
query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 36c15883-ea79-4a9e-ab7b-42f6864670b6, runId = be3f9b98-fded-46b5-927c-d347d96f0fb0] terminated with exception: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: describeTopics